In [2]:
# car detection and counting

import cv2 as cv
import numpy as np


min_w = 90
min_h = 90
line_high = 560
offset = 7
carno = 0
cars = []


def center(x,y,w,h):
    x1=int(w/2)
    y1=int(h/2)
    cx=x+x1
    cy=y+y1

    return cx, cy


cap = cv.VideoCapture("C:/Users/Administrator/PycharmProjects/vsCodeJupyterFiles/openCV/videos/highway6.mp4")

fgbg = cv.bgsegm.createBackgroundSubtractorMOG()
kernel = cv.getStructuringElement(cv.MORPH_RECT, (5,5))

while True:
    ret, frame = cap.read()

    if (ret == True):

        cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

        blur = cv.GaussianBlur(frame, (3,3), 5)
        mask = fgbg.apply(blur)

        erode = cv.erode(mask, kernel)
        dilate = cv.dilate(erode, kernel, iterations=3)

        close = cv.morphologyEx(dilate, cv.MORPH_CLOSE, kernel)
        close = cv.morphologyEx(close, cv.MORPH_CLOSE, kernel)

        cnts, h = cv.findContours(close, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

        cv.line(frame, (12,line_high), (1200,line_high), (255,255,10), 3)

        for (i, c) in enumerate(cnts):
            (x,y,w,h) = cv.boundingRect(c)

            isValid = (w >= min_w) and (h >= min_h)   # cette condition c'est pour capturer unikmnt les voitures
            if(not isValid):
                continue

            cv.rectangle((frame), (x,y), (x+w, y+h), (0,255,0), 3)
            cpoint = center(x,y,w,h)
            cars.append(cpoint)

            for (x,y) in cars:
                if ((y > line_high - offset) and (y < line_high + offset)):
                    carno += 1
                    cars.remove((x,y))
                    print(carno)

        

        cv.putText(frame, "Cars count:" + str(carno), (500,60), cv.FONT_HERSHEY_SIMPLEX, 2, (255,0,130), 5)
        cv.imshow('Video', frame)


        if cv.waitKey(5) & 0xFF == ord('q'):
            break


cap.release()
cv.destroyAllWindows()

1
2
3
4
5
6
7
